# Setup

In [2]:
# imports
import pandas as pd

In [4]:
# constants
RELEASED_VERSIONS = ["2.0.0", "2.0.1",
                     "2.1.0", "2.1.1", "2.2.0",
                     "2.3.0", "2.3.1", "2.3.2", "2.3.3", "2.3.4", "2.3.5", "2.3.6", "2.3.7", "2.3.8", "2.3.9",
                     "3.0.0", "3.1.0", "3.1.1", "3.1.2", "3.1.3"]

INPUT_DIR = "../../data/input/"
OUTPUT_DIR = "../../data/output/"

# Load data

We take a look at the raw metrics data to know how to process it 

In [31]:
df = pd.read_csv(INPUT_DIR + "hive-metrics-2.3.0.csv")
df.head()

,Kind,Name,AltAvgLineBlank,AltAvgLineCode,AltAvgLineComment,AltCountLineBlank,AltCountLineCode,AltCountLineComment,AvgCyclomatic,AvgCyclomaticModified,...,MaxInheritanceTree,MaxNesting,MinEssentialKnots,PercentLackOfCohesion,PercentLackOfCohesionModified,RatioCommentToCode,SumCyclomatic,SumCyclomaticModified,SumCyclomaticStrict,SumEssential
0,File,ACLConfigurationParser.java,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,...,NaN,3.0,NaN,NaN,NaN,0.16,29.0,29.0,32.0,22.0
1,File,AMReporter.java,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,3.0,NaN,NaN,NaN,0.13,65.0,65.0,68.0,36.0
2,File,ASTBuilder.java,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,...,NaN,2.0,NaN,NaN,NaN,0.09,90.0,32.0,90.0,22.0
3,File,ASTConverter.java,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,...,NaN,5.0,NaN,NaN,NaN,0.15,144.0,144.0,157.0,44.0
4,File,ASTErrorNode.java,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,0.0,NaN,NaN,NaN,0.95,5.0,5.0,5.0,5.0


We can see that we have metrics for not only files but also for packages, classes, and functions. Also, there are a lot more metrics than we want.
Therefore, we need to select precisely what we want from the dataset.

Below we can observe that classes' metrics don't have any value for files.

In [16]:
classes_metrics_cols = ["Name", "CountClassBase", "CountClassCoupled", "CountClassDerived", "MaxInheritanceTree", "PercentLackOfCohesion"]
df.loc[df["Kind"] == "File"][classes_metrics_cols].describe().loc["count"]

CountClassBase           0.0
CountClassCoupled        0.0
CountClassDerived        0.0
MaxInheritanceTree       0.0
PercentLackOfCohesion    0.0
Name: count, dtype: float64

Same for methods' metrics

In [18]:
methods_metrics_cols = ["Name", "CountInput", "CountClassCoupled", "CountClassDerived", "MaxInheritanceTree", "PercentLackOfCohesion"]
df.loc[df["Kind"] == "File"][methods_metrics_cols].describe().loc["count"]

CountInput               0.0
CountClassCoupled        0.0
CountClassDerived        0.0
MaxInheritanceTree       0.0
PercentLackOfCohesion    0.0
Name: count, dtype: float64

We therefore choose to only collect files' metrics as we are trying to predict bugs at files level.

In [30]:
metrics_cols = ["Name", "AvgCyclomatic", "AvgCyclomaticModified", "AvgCyclomaticStrict", "AvgEssential", "AvgLine", "AvgLineBlank", "AvgLineCode", 
    "AvgLineComment", "CountDeclClass", "CountDeclClassMethod", "CountDeclClassVariable", "CountDeclFunction", "CountDeclInstanceMethod",
    "CountDeclInstanceVariable", "CountDeclMethod", "CountDeclMethodDefault", "CountDeclMethodPrivate", "CountDeclMethodProtected",
    "CountDeclMethodPublic", "CountLine", "CountLineBlank", "CountLineCode", "CountLineCodeDecl", "CountLineCodeExe", "CountLineComment", 
    "CountSemicolon", "CountStmt", "CountStmtDecl", "CountStmtExe", "MaxCyclomatic", "MaxCyclomaticModified", "MaxCyclomaticStrict", 
    "RatioCommentToCode", "SumCyclomatic", "SumCyclomaticModified", "SumCyclomaticStrict", "SumEssential"]

data = df.loc[df["Kind"] == "File"][metrics_cols]
data.describe()

,AvgCyclomatic,AvgCyclomaticModified,AvgCyclomaticStrict,AvgEssential,AvgLine,AvgLineBlank,AvgLineCode,AvgLineComment,CountDeclClass,CountDeclClassMethod,...,CountStmtDecl,CountStmtExe,MaxCyclomatic,MaxCyclomaticModified,MaxCyclomaticStrict,RatioCommentToCode,SumCyclomatic,SumCyclomaticModified,SumCyclomaticStrict,SumEssential
count,7646.000000,7646.000000,7646.000000,7646.000000,7646.000000,7646.000000,7646.000000,7646.000000,7646.000000,7616.000000,...,7646.000000,7646.000000,7646.000000,7646.000000,7646.000000,7646.000000,7646.000000,7646.000000,7646.000000,7646.000000
mean,1.715799,1.609338,1.863327,1.050353,11.888046,0.638373,9.965995,0.911849,2.173816,1.541492,...,59.714491,97.519749,5.767722,5.284070,7.114177,0.823925,36.262621,34.314020,38.971096,20.974496
std,1.617329,1.381750,1.860476,0.715302,12.002911,1.768376,9.618689,1.988870,14.232159,10.333067,...,308.970481,857.965920,8.147524,7.269429,11.569944,1.946497,250.694914,236.279118,254.760264,132.214127
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,1.000000,5.000000,0.000000,4.000000,0.000000,1.000000,0.000000,...,13.000000,7.000000,1.000000,1.000000,1.000000,0.210000,4.000000,4.000000,4.000000,3.000000
50%,1.000000,1.000000,1.000000,1.000000,9.000000,0.000000,7.000000,0.000000,1.000000,0.000000,...,27.000000,25.000000,3.000000,3.000000,3.000000,0.400000,10.000000,10.000000,10.000000,7.000000
75%,2.000000,2.000000,2.000000,1.000000,16.000000,1.000000,13.000000,1.000000,2.000000,1.000000,...,61.000000,75.000000,7.000000,6.000000,8.000000,0.797500,27.000000,26.000000,29.000000,16.000000
max,33.000000,21.000000,33.000000,13.000000,354.000000,66.000000,260.000000,28.000000,1093.000000,743.000000,...,17958.000000,64320.000000,126.000000,116.000000,204.000000,66.000000,17006.000000,15976.000000,17006.000000,7702.000000


In [23]:
data.insert(1, "Version", "2.0.0")
data.head()

,Name,Version,AvgCyclomatic,AvgCyclomaticModified,AvgCyclomaticStrict,AvgEssential,AvgLine,AvgLineBlank,AvgLineCode,AvgLineComment,...,CountStmtDecl,CountStmtExe,MaxCyclomatic,MaxCyclomaticModified,MaxCyclomaticStrict,RatioCommentToCode,SumCyclomatic,SumCyclomaticModified,SumCyclomaticStrict,SumEssential
1,ACIDBenchmarks.java,2.0.0,1.0,1.0,1.0,1.0,6.0,0.0,6.0,0.0,...,75.0,66.0,5.0,5.0,5.0,0.08,34.0,34.0,34.0,23.0
2,ACLConfigurationParser.java,2.0.0,2.0,2.0,3.0,2.0,10.0,0.0,10.0,0.0,...,37.0,58.0,11.0,11.0,13.0,0.16,29.0,29.0,32.0,22.0
3,AMReporter.java,2.0.0,1.0,1.0,1.0,1.0,11.0,0.0,10.0,0.0,...,168.0,196.0,7.0,7.0,9.0,0.12,77.0,77.0,81.0,43.0
4,ANTLRNoCaseStringStream.java,2.0.0,2.0,2.0,2.0,2.0,7.0,1.0,6.0,0.0,...,7.0,7.0,3.0,3.0,3.0,1.72,4.0,4.0,4.0,4.0
5,ASTBuilder.java,2.0.0,5.0,1.0,5.0,1.0,18.0,0.0,16.0,1.0,...,71.0,143.0,72.0,10.0,72.0,0.11,107.0,38.0,107.0,24.0


We have a proper dataset to work on. 

Now, let us replicate this process on all versions of files.

In [ ]:
for version in RELEASED_VERSIONS:
    df = pd.read_csv(INPUT_DIR + f"hive-metrics-{version}.csv")
    df = df.loc[df["Kind"] == "File"][metrics_cols]
    df.insert(1, "Version", version)
    df.to_csv(OUTPUT_DIR + f"hive-files-metrics-{version}.csv", index=False)
    

We can now concatenate all the generated files above, to have the unlabeled dataset.

In [ ]:
versions_data = []

for version in RELEASED_VERSIONS:
    df = pd.read_csv(OUTPUT_DIR + f"hive-files-metrics-{version}.csv")
    versions_data.append(df)

unlabeled_data = pd.concat(versions_data)
unlabeled_data.to_csv(OUTPUT_DIR + f"hive-unlabeled-data.csv", index=False)